<a href="https://colab.research.google.com/github/TerrorismAnalyticsBureau/TAB-AI/blob/main/TABAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim
import matplotlib.pyplot as plt
import math
import os
import chardet
from sklearn.preprocessing import LabelEncoder

# Set pyTorch local env to use segmented GPU memory
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Clear GPU cache & Set the device to use GPU
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
# Skip rows = 1 because those are the column names
X = np.array([])


# Load the data
# Open the file and detect the encoding
#with open('./globalterrorismdb_0718dist.csv', 'rb') as f:
#    result = chardet.detect(f.read())

# Get the detected encoding
#print(f"Detected encoding: {result['encoding']}")

# Now, read the file using the detected encoding
data = pd.read_csv('./globalterrorismdb_0718dist.csv', encoding="Windows-1252")

# Extract relevant columns (adjust indices or column names as needed)
input_columns = data.iloc[:, [1, 2, 3, 7, 9]]
input_columns = input_columns.fillna(0)

attack_target = data.iloc[:, [28]]
group_target = data.iloc[:, [57]]
date_target = data.iloc[:, [1, 2, 3]]

# Extract unique group names
unique_groups = list(set(data['gname']))
unique_provstates = list(set(data['provstate']))
unique_cities = list(set(data['city']))

# Initialize LabelEncoder and fit to the unique groups
label_encoder = LabelEncoder()
label_encoder.fit(unique_groups)
label_encoder.fit(unique_provstates)
label_encoder.fit(unique_cities)

# Create a dictionary to map group names to their encoded IDs
groupsDict = {group: label_encoder.transform([group])[0] for group in unique_groups}
provstatesDict = {provstate: label_encoder.transform([provstate])[0] for provstate in unique_provstates}
citiesDict = {city: label_encoder.transform([city])[0] for city in unique_cities}

# Assign values to tensors for processing
input_tensor = torch.tensor(input_columns.values, dtype=torch.float32)
attack_target_tensor = torch.tensor(attack_target.values, dtype=torch.float32)
group_target_tensor = torch.tensor(label_encoder.fit_transform(group_target.values), dtype=torch.float32)
date_target_tensor = torch.tensor(date_target.values, dtype=torch.float32)

# TESTING -
for key, value in groupsDict.items():
  print("group: ", {key}, "group #:", {value})

# use cols skips over column 6 as that is the symbol of the stock
X_tensor = input_tensor
Y_tensor_attack = attack_target_tensor
Y_tensor_group = group_target_tensor
Y_tensor_date = date_target_tensor

# Set tensors to use GPU
X_tensor = X_tensor.to(device)
Y_tensor_attack = Y_tensor_attack.to(device)
Y_tensor_group = Y_tensor_group.to(device)
Y_tensor_date = Y_tensor_date.to(device)

class LSTMAttackPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMAttackPredictor, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])  # Use the last time step output
        return out


model = LSTMAttackPredictor(input_size=5, hidden_size=128, output_size=3)
model  = model.to(device)


<ipython-input-5-dbde663b2124>:34: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./globalterrorismdb_0718dist.csv', encoding="Windows-1252")


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

name (csv column number)
```
•    iyear (1), imonth (2), iday (3)
•    country code (7) and country_txt (8)
•    region code (9) and region_txt (10)
•    provstate (11) and city (12)
•    latitude (13) and longitude (14)
•    attacktype1 (28) and attacktype1_txt (29)
•    targtype1 (34) and targtype1_txt (35)
•    targsubtype1 (36) and targsubtype1_txt (37)
•    target1 (39) (the specific target by name, building or person)

•    natity1 (40) and natity1_txt (41) (maybe later)
•    gname (group name) (57)
•    weaptype1 (81) and weaptype1_txt (82) (maybe)
•    nkill (98) and nwound (101)

```

